In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# --- Load inflow data ---
# Current CSV link from USBR dashboard
inflow_url = "https://www.usbr.gov/uc/water/hydrodata/crmms/current/1_2025/919/csv/30.csv"
inflow_df = pd.read_csv(inflow_url)

# Ensure 'date' is datetime
inflow_df['date'] = pd.to_datetime(inflow_df['date'])

# Select just the date and minimum probable inflow
min_prob_df = inflow_df[['date', '24MS MIN PROB']]
pd.set_option('display.max_rows', None)
print(min_prob_df)

# --- Compute 12‑month totals ---
values = inflow_df['24MS MIN PROB']

# First 12 months
first_12_sum_af = values.iloc[:12].sum()
first_12_sum_maf = first_12_sum_af / 1_000_000

# Second 12 months
second_12_sum_af = values.iloc[12:24].sum()
second_12_sum_maf = second_12_sum_af / 1_000_000

# --- Scenario table ---
data = {
    "Trace": [
        "Glen Canyon Dam Release",
        "Unregulated Inflow into Lake Powell",
        "Immersive Model Scenario 1",
        "Immersive Model Scenario 2"
    ],
    "2025": [7.481, 4.735, 2.8, 4.7],  #Expand where the forecast for the USBR inflow came from
    "2026": [7.480, first_12_sum_maf, 4.7, 3.7],
    "2027": [7.480, second_12_sum_maf, 4.2, 3.4]
}

df = pd.DataFrame(data)
print(df)

# --- Consecutive 12‑month block totals ---
inflow_df['min_prob_maf'] = inflow_df['24MS MIN PROB'] / 1_000_000

block_totals = []
for i in range(0, len(inflow_df), 12):
    block = inflow_df.iloc[i:i+12]
    if len(block) == 12:  # only full 12‑month blocks
        start = block['date'].iloc[0].strftime("%b %Y")
        end = block['date'].iloc[-1].strftime("%b %Y")
        label = f"{start} – {end}"
        total = block['min_prob_maf'].sum()
        block_totals.append({"Period": label, "Total MAF": total})

block_df = pd.DataFrame(block_totals)
print(block_df)



         date  24MS MIN PROB
0  2024-12-01     348916.353
1  2025-01-01     353498.008
2  2025-02-01     328780.489
3  2025-03-01     429988.135
4  2025-04-01     431332.043
5  2025-05-01     833714.023
6  2025-06-01    1111944.190
7  2025-07-01     563595.688
8  2025-08-01     410521.231
9  2025-09-01     359097.830
10 2025-10-01     403242.188
11 2025-11-01     410833.418
12 2025-12-01     351967.927
13 2026-01-01     338786.980
14 2026-02-01     377899.489
15 2026-03-01     503187.135
16 2026-04-01     609864.043
17 2026-05-01    1317646.020
18 2026-06-01    1250811.190
19 2026-07-01     423320.688
20 2026-08-01     371882.231
21 2026-09-01     354471.830
22 2026-10-01     399635.188
23 2026-11-01     402953.418
24 2026-12-01     384747.927
                                 Trace   2025      2026      2027
0              Glen Canyon Dam Release  7.481  7.480000  7.480000
1  Unregulated Inflow into Lake Powell  4.735  5.985464  6.702426
2           Immersive Model Scenario 1  2.800  4